In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans #, k_means
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('./data/mlb-pitches.csv')
data.drop(columns = ['Unnamed: 0'], inplace = True) 

pd.set_option('max_columns', None)

print(data.shape)
#data.head(3)
#list(data.columns.values)
#data.info()

(709851, 86)


In [3]:
df = data[['pitcher', 'p_throws', 'pitch_type', 'release_speed', 
           'release_spin_rate', 'spin_axis', 'pfx_-x', 'pfx_z', 
           'release_extension', 'release_pos_x', 'release_pos_z', 'effective_speed', 
           'plate_x', 'plate_-x', 'plate_z', 'vx0', 'vy0', 'vz0', 
           'ax', 'ay', 'az', 'type', 'zone', 'balls', 'strikes', 
           'batter', 'stand', 'events', 'description', 'hit_location', 
           'bb_type', 'hc_x','hc_y', 'hit_distance_sc', 'launch_speed', 
           'launch_angle', 'launch_speed_angle', 'estimated_ba_using_speedangle', 
           'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 
           'babip_value', 'iso_value','outs_when_up', 'inning', 'game_pk', 
           'release_pos_y', 'if_fielding_alignment', 'of_fielding_alignment', 
           'pitch_number']].copy()

print(df.shape)
#df.head()

(709851, 50)


In [4]:
#df[df.columns[df.isnull().any()]].isnull().sum()
#df.dropna(inplace = True)
df.dropna(subset = ['release_extension', 'release_spin_rate', 'if_fielding_alignment'], 
          inplace = True)

# Fill nulls with none so they aren't dropped
df['bb_type'] = df['bb_type'].fillna('not_in_play').astype(str)
df['events'] = df['events'].fillna('none').astype(str)
df.fillna(value = 0, inplace = True)


df = pd.get_dummies(df, columns = ['p_throws', 'type', 'stand', 'events', 
                                   'description', 'bb_type', 'if_fielding_alignment', 
                                   'of_fielding_alignment'])

# Drop pitch types Fasball, knuckleball, eephus, and screwball
df['pitch_type'] = df['pitch_type'].map({'SC': 12, 'EP': 11, 'KN': 10, 'FA': 9, 
                                         'FS': 8, 'CH': 7, 'KC': 6, 'CU': 5, 
                                         'SL': 4, 'FC': 3, 'SI': 2, 'FF': 1})
df = df.loc[df['pitch_type'] < 9]

print(df.shape)
df.head()

(702443, 109)


,pitcher,pitch_type,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,effective_speed,plate_x,plate_-x,plate_z,vx0,vy0,vz0,ax,ay,az,zone,balls,strikes,batter,hit_location,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,launch_speed_angle,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,outs_when_up,inning,game_pk,release_pos_y,pitch_number,p_throws_L,p_throws_R,type_B,type_S,type_X,stand_L,stand_R,events_catcher_interf,events_caught_stealing_2b,events_caught_stealing_3b,events_caught_stealing_home,events_double,events_double_play,events_field_error,events_field_out,events_fielders_choice,events_fielders_choice_out,events_force_out,events_game_advisory,events_grounded_into_double_play,events_hit_by_pitch,events_home_run,events_none,events_other_out,events_passed_ball,events_pickoff_1b,events_pickoff_2b,events_pickoff_3b,events_pickoff_caught_stealing_2b,events_pickoff_caught_stealing_3b,events_sac_bunt,events_sac_bunt_double_play,events_sac_fly,events_sac_fly_double_play,events_single,events_stolen_base_2b,events_strikeout,events_strikeout_double_play,events_triple,events_triple_play,events_walk,events_wild_pitch,description_ball,description_blocked_ball,description_bunt_foul_tip,description_called_strike,description_foul,description_foul_bunt,description_foul_pitchout,description_foul_tip,description_hit_by_pitch,description_hit_into_play,description_missed_bunt,description_pitchout,description_swinging_strike,description_swinging_strike_blocked,bb_type_fly_ball,bb_type_ground_ball,bb_type_line_drive,bb_type_not_in_play,bb_type_popup,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_4th outfielder,of_fielding_alignment_Standard,of_fielding_alignment_Strategic
0,519293,1.0,92.3,2330.0,148.0,-8.28,16.56,6.5,1.40,6.80,92.8,-0.69,0.69,2.83,-6.833043,-134.166485,-7.361843,9.708393,26.562803,-14.083224,1.0,1,2,596019,5.0,77.54,155.31,13.0,95.2,-13.0,2.0,0.174,0.158,0.0,1.0,0.0,0.0,2,9,632254,54.03,4,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
1,519293,4.0,80.6,2254.0,315.0,9.24,5.76,6.4,1.60,6.64,81.2,-0.71,0.71,2.62,-3.700232,-117.430885,-3.266842,-6.531123,19.793390,-27.369114,4.0,1,1,596019,0.0,0.00,0.00,108.0,75.3,75.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,2,9,632254,54.15,3,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
2,519293,5.0,75.5,1940.0,328.0,7.80,-6.12,6.2,1.46,6.88,75.2,-0.04,0.04,2.46,-1.977183,-109.901781,-1.155694,-4.872924,20.602334,-36.262184,5.0,1,0,596019,0.0,0.00,0.00,157.0,83.5,65.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,2,9,632254,54.34,2,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
3,519293,5.0,75.0,2017.0,330.0,8.28,-8.28,5.9,1.53,6.83,74.5,2.10,-2.10,3.89,2.375830,-109.205830,2.277617,-5.902656,19.427562,-38.284747,12.0,0,0,596019,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,2,9,632254,54.61,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
4,519293,1.0,91.2,2281.0,143.0,-7.56,15.36,6.3,1.49,6.66,90.9,-0.31,0.31,2.80,-5.868477,-132.500539,-6.486796,8.700586,30.117690,-15.941174,4.0,1,0,607043,4.0,144.83,145.72,9.0,93.3,-18.0,2.0,0.100,0.090,0.0,1.0,0.0,0.0,1,9,632254,54.15,2,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 702443 entries, 0 to 709849
Columns: 109 entries, pitcher to of_fielding_alignment_Strategic
dtypes: float64(34), int64(8), uint8(67)
memory usage: 275.3 MB


# Pitch Numbers

1. 4-seam fastball
2. Sinker
3. Cutter
4. Slider
5. Curveball
6. Knuckle Curve
7. Changeup
8. Split-Finger

### # of Pitches

In [6]:
df.pitch_type.value_counts()

1.0    248734
4.0    135469
2.0    108687
7.0     79982
5.0     58582
3.0     46581
6.0     13586
8.0     10822
Name: pitch_type, dtype: int64

### Velocity

In [7]:
df.groupby('pitch_type')['release_speed'].mean()

pitch_type
1.0    93.701754
2.0    93.002663
3.0    88.532528
4.0    84.669771
5.0    78.737820
6.0    81.071603
7.0    84.815686
8.0    85.706644
Name: release_speed, dtype: float64

### Spin Rate

In [8]:
df.groupby('pitch_type')['release_spin_rate'].mean()

pitch_type
1.0    2273.265428
2.0    2123.280760
3.0    2371.815547
4.0    2416.469052
5.0    2506.390530
6.0    2536.233770
7.0    1749.887312
8.0    1365.298743
Name: release_spin_rate, dtype: float64

### Spin Axis

In [9]:
df.groupby('pitch_type')['spin_axis'].mean()

pitch_type
1.0    192.877624
2.0    194.736095
3.0    174.086065
4.0    146.045368
5.0    124.552559
6.0     77.354924
7.0    195.125828
8.0    227.108575
Name: spin_axis, dtype: float64

### HB

In [10]:
df.groupby('pitch_type')['pfx_-x'].mean()

pitch_type
1.0    2.852320
2.0    5.644772
3.0   -1.369132
4.0   -3.203078
5.0   -3.904535
6.0   -6.285912
7.0    3.620699
8.0    9.603515
Name: pfx_-x, dtype: float64

### VB

In [11]:
df.groupby('pitch_type')['pfx_z'].mean()

pitch_type
1.0    16.101475
2.0     9.225868
3.0     7.946050
4.0     1.712970
5.0    -8.714744
6.0   -11.299573
7.0     6.825582
8.0     3.874884
Name: pfx_z, dtype: float64

### Pitch Data for Model

In [12]:
pitch_data = df[['pitch_type', 'release_speed', 'release_spin_rate', 'spin_axis', 
                 'pfx_-x', 'pfx_z','release_extension', 'release_pos_x', 'release_pos_z']]

pitch_data.dropna(inplace = True)

print(pitch_data.shape)
pitch_data.head()

(702443, 9)


,pitch_type,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z
0,1.0,92.3,2330.0,148.0,-8.28,16.56,6.5,1.40,6.80
1,4.0,80.6,2254.0,315.0,9.24,5.76,6.4,1.60,6.64
2,5.0,75.5,1940.0,328.0,7.80,-6.12,6.2,1.46,6.88
3,5.0,75.0,2017.0,330.0,8.28,-8.28,5.9,1.53,6.83
4,1.0,91.2,2281.0,143.0,-7.56,15.36,6.3,1.49,6.66


## Model

In [13]:
X = pitch_data

ss = StandardScaler()
X_scaled = ss.fit_transform(X)
X_scaled[:,:] *= -1

pca = PCA().fit(X_scaled)
principal_components = pca.transform(X_scaled)

In [14]:
model = pd.DataFrame(data = principal_components, columns = X.columns)

print(model.shape)
model.head()

(702443, 9)


,pitch_type,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z
0,-0.701653,-2.169353,-1.194179,0.116000,-0.980121,0.540040,0.086666,0.199248,0.085256
1,0.082593,1.152907,-1.582649,0.862290,-0.403320,2.210316,0.538640,0.624039,0.192911
2,1.334992,2.098575,-2.211249,1.256269,-0.622173,2.074772,-0.128482,1.169299,0.654585
3,1.565962,2.113372,-2.133282,1.827637,-0.189419,2.013466,-0.207566,1.150013,0.645387
4,-0.487812,-2.013401,-1.237600,0.295428,-0.576891,0.309730,0.005585,0.325436,-0.034896


In [15]:
km = KMeans(n_clusters = 4,
            random_state = 0)

km.fit(model)
label = km.fit_predict(model)

print(label)

print('label:', km.labels_)
print('Number of iterations:', km.n_iter_)
print('Number of features:', km.n_features_in_)
print('Number of clusters:', km.n_clusters)
print('Inertia:', km.inertia_)
#print('Silhouette Score:', silhouette_score(model, label))

[2 3 3 ... 0 0 0]
label: [2 3 3 ... 0 0 0]
Number of iterations: 11
Number of features: 9
Number of clusters: 4
Inertia: 3404596.538809984


In [16]:
centroids = km.cluster_centers_
print("Predicted clusters to points: ", label[:10], '\n')
print('Location of centroids:', '\n')
print(centroids)

Predicted clusters to points:  [2 3 3 3 2 2 3 3 2 0] 

Location of centroids: 

[[-1.55336592  0.10036944  0.43477987  0.1527813  -0.13472579 -0.09139098
  -0.02853423  0.0634657   0.01193692]
 [ 2.24309223 -0.46916117  1.0566219   0.13303272 -0.40868491 -0.24953026
  -0.04901329  0.05319106 -0.06565297]
 [ 0.25457638 -1.22582023 -1.19289473 -0.50534952  0.45202868 -0.02102278
  -0.01446046 -0.07699617 -0.03744402]
 [ 0.58467342  2.09824964 -0.67963153  0.18401427  0.19509421  0.56571964
   0.15156775 -0.11090844  0.10711965]]


### Cluster Labels

Label:

0. Fastballs (FF, SI, FC)
1. Breaking Balls (SL, CU, **KC**)
2. FB's & BB's (FC, SL, CU)
3. Changeups (CH, Splitter)

In [17]:
X['label'] = label
model['label'] = label

#model.groupby(by = 'label').mean().T
X.groupby(by = 'label').mean().T

label,0,1,2,3
pitch_type,1.504309,4.396868,2.711462,5.820105
release_speed,93.580099,82.794899,89.940428,83.619479
release_spin_rate,2259.085899,2500.921407,2121.634666,2004.913479
spin_axis,211.932981,74.696864,143.159597,261.596818
pfx_-x,8.244117,-7.918710,-8.726993,10.303884
pfx_z,13.681498,-2.789637,11.675644,1.779559
release_extension,6.376585,6.268359,6.260769,6.250175
release_pos_x,-1.797669,-1.810608,1.925868,-0.420718
release_pos_z,5.863410,5.825122,5.917093,5.787879
